## Data Preprocessing

In [2]:
import pandas as pd
from process import read_data

data_folder = "../data/earning/"
years = range(2011, 2025)
target_col = ['REGULAR','RETRO','OTHER','OVERTIME','INJURED','DETAIL','QUINN_EDUCATION','TOTAL_GROSS']
drop_col = ['_ID', 'POSTAL']
encoding = 'ISO-8859-1'

dataframes = read_data(data_folder, 'earnings.csv', years, encoding, target_col, drop_col)

Read data from: ../data/earning/2011_earnings.csv
Read data from: ../data/earning/2012_earnings.csv
Read data from: ../data/earning/2013_earnings.csv
Read data from: ../data/earning/2014_earnings.csv
Read data from: ../data/earning/2015_earnings.csv
Read data from: ../data/earning/2016_earnings.csv
Read data from: ../data/earning/2017_earnings.csv
Read data from: ../data/earning/2018_earnings.csv
Read data from: ../data/earning/2019_earnings.csv
Read data from: ../data/earning/2020_earnings.csv
Read data from: ../data/earning/2021_earnings.csv
Read data from: ../data/earning/2022_earnings.csv
Read data from: ../data/earning/2023_earnings.csv
Read data from: ../data/earning/2024_earnings.csv
Successfully read data from all files.


In [3]:
# Concatenate all dataframes into one
full_df = pd.concat(dataframes, ignore_index=True)

#print the data type of each column
print("\nData Types:")
print(full_df.dtypes)

# display random 10 rows from 2023
print("\nSample from 2012:")
display(full_df[full_df['YEAR'] == 2012].sample(10))


Data Types:
NAME                object
DEPARTMENT_NAME     object
TITLE               object
REGULAR            float64
RETRO              float64
OTHER              float64
OVERTIME           float64
INJURED            float64
DETAIL             float64
QUINN_EDUCATION    float64
TOTAL_GROSS        float64
YEAR                 int64
dtype: object

Sample from 2012:


,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN_EDUCATION,TOTAL_GROSS,YEAR
27832,"Lee,Waiman",Boston Police Department,Police Detective,74743.65,NaN,11753.52,60640.81,NaN,59850.0,9343.06,216331.04,2012
34283,"Valencia,Shannon D",Boston Public Schools,Teacher,84977.32,NaN,NaN,NaN,NaN,NaN,NaN,84977.32,2012
36658,"Garvey,Julie McNulty",Boston Public Schools,Paraprofessional,31518.93,NaN,NaN,NaN,NaN,NaN,NaN,31518.93,2012
32595,"James,Shana",Boston Public Schools,Paraprofessional,665.13,NaN,NaN,NaN,NaN,NaN,NaN,665.13,2012
40233,"Sadberry,Lisa M",Boston Public Schools,Social Worker/Coordinator,85973.00,NaN,NaN,NaN,NaN,NaN,NaN,85973.00,2012
26398,"Boissaye,Rebecca",Boston Police Department,Criminalist II,57413.29,475.39,1366.84,2651.83,NaN,NaN,NaN,61907.35,2012
25198,"Glora,Paul G",Boston Fire Department,Fire Captain,119452.72,NaN,550.00,14270.90,NaN,NaN,NaN,134273.62,2012
27324,"Golden,Kevin F",Boston Police Department,Police Officer,67934.43,NaN,800.00,9091.98,246.81,9713.0,NaN,87786.22,2012
40786,"Corcoran,Victoria S.",Boston Public Schools,Substitute Teacher,11732.80,NaN,NaN,NaN,NaN,NaN,NaN,11732.80,2012
22239,"Foley,Mary A",Dpt of Innovation & Technology,Sr Data Proc Sys Anl,86606.78,NaN,NaN,NaN,NaN,NaN,NaN,86606.78,2012


In [4]:
# 1. Compute annual summaries per department (summing TOTAL_GROSS and OVERTIME)
department_summaries = {}
for year, df_year in full_df.groupby('YEAR'):
    summary = (
        df_year
        .groupby('DEPARTMENT_NAME')[['TOTAL_GROSS', 'OVERTIME']]
        .sum()
        .reset_index()
    )
    department_summaries[year] = summary

# 2. For each year, select the top 5 departments by overtime pay
top_departments = {
    year: df.nlargest(5, 'OVERTIME')['DEPARTMENT_NAME'].tolist()
    for year, df in department_summaries.items()
}

# 3. Build yearly summary tables that include only those top 5 departments
top_department_data = {
    year: df[df['DEPARTMENT_NAME'].isin(top_departments[year])].copy()
    for year, df in department_summaries.items()
}

# 4. Find departments that appear in the top 5 every year
years_list = sorted(top_departments.keys())
common_departments = set(top_departments[years_list[0]])
for year in years_list[1:]:
    common_departments &= set(top_departments[year])

print(f"Departments in the overtime top 5 every year: {common_departments}")

# 5. For each of those ‘consistent’ departments, concatenate their yearly data
department_dataframes = {}
for dept in common_departments:
    pieces = []
    for year, df in top_department_data.items():
        sub = df[df['DEPARTMENT_NAME'] == dept]
        if not sub.empty:
            sub = sub.copy()
            sub['YEAR'] = year
            pieces.append(sub)
    if pieces:
        department_dataframes[dept] = pd.concat(pieces, ignore_index=True)

# 6. Display each department’s DataFrame
for dept, df in department_dataframes.items():
    print(f"\n===== Department: {dept} =====")
    display(df)

Departments in the overtime top 5 every year: {'Public Works Department', 'Property Management', 'Boston Police Department', 'Boston Fire Department'}

===== Department: Public Works Department =====


,DEPARTMENT_NAME,TOTAL_GROSS,OVERTIME,YEAR
0,Public Works Department,21637431.56,2936045.81,2011
1,Public Works Department,20839543.57,1727033.69,2012
2,Public Works Department,22160388.68,3149769.31,2013
3,Public Works Department,24888293.92,3833240.06,2014
4,Public Works Department,27029986.10,5243898.07,2015
5,Public Works Department,25931149.01,4161927.40,2016
6,Public Works Department,26674448.16,5114656.36,2017
7,Public Works Department,27067680.29,5255554.84,2018
8,Public Works Department,26285862.95,4506512.51,2019
9,Public Works Department,26098861.91,3444542.88,2020



===== Department: Property Management =====


,DEPARTMENT_NAME,TOTAL_GROSS,OVERTIME,YEAR
0,Property Management,14173062.33,1509684.21,2011
1,Property Management,14727492.81,1771431.66,2012
2,Property Management,15225876.72,2136854.78,2013
3,Property Management,15103896.30,1948891.72,2014
4,Property Management,14867023.90,2131552.41,2015
5,Property Management,10103922.14,1814680.76,2016
6,Property Management,10570764.93,1917753.05,2017
7,Property Management,11080659.67,1943773.68,2018
8,Property Management,11006087.59,2075398.70,2019
9,Property Management,10775455.78,2506471.06,2020



===== Department: Boston Police Department =====


,DEPARTMENT_NAME,TOTAL_GROSS,OVERTIME,YEAR
0,Boston Police Department,2.905179e+08,4.223736e+07,2011
1,Boston Police Department,2.958616e+08,4.469742e+07,2012
2,Boston Police Department,3.077962e+08,5.748377e+07,2013
3,Boston Police Department,3.578099e+08,5.791461e+07,2014
4,Boston Police Department,3.581379e+08,6.160814e+07,2015
5,Boston Police Department,3.885872e+08,6.098992e+07,2016
6,Boston Police Department,3.912776e+08,6.693365e+07,2017
7,Boston Police Department,4.166830e+08,7.785544e+07,2018
8,Boston Police Department,4.157256e+08,7.776430e+07,2019
9,Boston Police Department,4.162761e+08,7.805770e+07,2020



===== Department: Boston Fire Department =====


,DEPARTMENT_NAME,TOTAL_GROSS,OVERTIME,YEAR
0,Boston Fire Department,1.731703e+08,20940928.99,2011
1,Boston Fire Department,1.791011e+08,21408244.96,2012
2,Boston Fire Department,1.778649e+08,20388822.87,2013
3,Boston Fire Department,2.097233e+08,21318731.45,2014
4,Boston Fire Department,2.094536e+08,24815833.23,2015
5,Boston Fire Department,2.114572e+08,23762982.32,2016
6,Boston Fire Department,2.151573e+08,22829249.92,2017
7,Boston Fire Department,2.199342e+08,23679890.59,2018
8,Boston Fire Department,2.655112e+08,30274873.99,2019
9,Boston Fire Department,2.642356e+08,33761132.79,2020


## Overtime Analysis

In [5]:
import plotly.graph_objects as go

def plot_department_overtime_interactive(department_dataframes):
    """
    department_dataframes: dict
      key   = department name (str)
      value = DataFrame with 'YEAR' and 'OVERTIME' columns
    """
    # Create an empty Figure
    fig = go.Figure()

    # Add one line trace per department
    for dept, df in department_dataframes.items():
        df_sorted = df.sort_values('YEAR')
        fig.add_trace(
            go.Scatter(
                x=df_sorted['YEAR'],
                y=df_sorted['OVERTIME'],
                mode='lines+markers',
                name=dept,
                hovertemplate=
                    '<b>%{text}</b><br>' +
                    'Year: %{x}<br>' +
                    'Overtime Pay: $%{y:,.2f}<extra></extra>',
                text=[dept] * len(df_sorted)
            )
        )

    # Update layout for titles and axes
    fig.update_layout(
        title="Overtime Pay Trends for Consistently Top Departments (2011–2024)",
        xaxis_title="Year",
        yaxis_title="Overtime Pay (USD)",
        xaxis=dict(tickmode='linear'),
        template='plotly_white',
        legend_title="Department",
        hovermode='x unified'
    )

    fig.show()

# Example call
plot_department_overtime_interactive(department_dataframes)

## BPD Overtime Forecast

In [11]:
from regression import run_overtime_forecast

results = run_overtime_forecast(
    department_name='Boston Police Department',
    dept_dfs=department_dataframes,
    forecast_years=2,
    test_degrees=[2, 3, 4],
    interactive=True
)

Best polynomial degree: 4

Predicted future values:
Year 2025: $109,631,422.77
Year 2026: $123,230,936.16

Best model equation:
y = -1.23e+14 + 1.16e+05x^1 + 1.81e+08x^2 + -1.19e+05x^3 + 2.22e+01x^4



=== Forecast Summary for Boston Police Department ===
* Best polynomial degree: 4
* R² score             : 0.9301
* Model equation       : y = -1.23e+14 + 1.16e+05x^1 + 1.81e+08x^2 + -1.19e+05x^3 + 2.22e+01x^4


## BFD Overtime Forecast

In [14]:
results = run_overtime_forecast(
    department_name='Boston Fire Department',
    dept_dfs=department_dataframes,
    forecast_years=2,
    test_degrees=[2, 3, 4],
    interactive=True
)

Best polynomial degree: 4

Predicted future values:
Year 2025: $47,546,064.46
Year 2026: $49,219,290.30

Best model equation:
y = 3.87e+13 + -3.65e+04x^1 + -5.69e+07x^2 + 3.76e+04x^3 + -6.97e+00x^4



=== Forecast Summary for Boston Fire Department ===
* Best polynomial degree: 4
* R² score             : 0.9442
* Model equation       : y = 3.87e+13 + -3.65e+04x^1 + -5.69e+07x^2 + 3.76e+04x^3 + -6.97e+00x^4


## Property Management Overtime Forecast

In [16]:
results = run_overtime_forecast(
    department_name="Property Management",
    dept_dfs=department_dataframes,
    forecast_years=2,
    test_degrees=[2, 3, 4],
    interactive=True
)

Best polynomial degree: 3

Predicted future values:
Year 2025: $3,140,879.32
Year 2026: $3,493,237.97

Best model equation:
y = -1.06e+13 + 1.58e+10x^1 + -7.82e+06x^2 + 1.29e+03x^3



=== Forecast Summary for Property Management ===
* Best polynomial degree: 3
* R² score             : 0.8246
* Model equation       : y = -1.06e+13 + 1.58e+10x^1 + -7.82e+06x^2 + 1.29e+03x^3


## Public Works Department Overtime Forecast

In [17]:
results = run_overtime_forecast(
    department_name='Public Works Department',
    dept_dfs=department_dataframes,
    forecast_years=2,
    test_degrees=[2, 3, 4],
    interactive=True
)

Best polynomial degree: 4

Predicted future values:
Year 2025: $7,973,137.94
Year 2026: $9,883,230.89

Best model equation:
y = -2.24e+13 + 2.11e+04x^1 + 3.30e+07x^2 + -2.18e+04x^3 + 4.05e+00x^4



=== Forecast Summary for Public Works Department ===
* Best polynomial degree: 4
* R² score             : 0.7005
* Model equation       : y = -2.24e+13 + 2.11e+04x^1 + 3.30e+07x^2 + -2.18e+04x^3 + 4.05e+00x^4
